In [1]:
%matplotlib tk 
import mne
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/media/mangaldeep/HDD2/workspace/MotionControl_MasterThesis')
from main.extraction.physionet_MI import extractPhysionet
from data import brain_atlas  as bm

In [2]:
raw = extractPhysionet(runs = [3], person_id = 3)
# raw.pick_channels(['C3','C4'])
raw.notch_filter(60)
raw = raw.filter(7,50,verbose = False).copy()

event_data = mne.events_from_annotations(raw)
event_marker, event_ids = event_data
event_ids = dict({'T1':2, 'T2':3})
epochs = mne.Epochs(raw, event_marker, event_ids, tmin=-2, tmax=4, preload = True)

Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
Not setting metadata
15 matching events found
Setting baseline interval to [-2.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 15 events and 961 original time points ...
0 bad epochs dropped


In [3]:
epoch_file = '../preproc/Physionet_trial_epo.fif'
epochs = mne.read_epochs(epoch_file)

Reading ../preproc/Physionet_trial_epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
15 matching events found
No baseline correction applied
0 projection items activated


In [3]:
epochs = epochs.apply_proj().copy()
epochs = epochs.apply_baseline((-2.0,-0.2)).copy()
epochs.equalize_event_counts()
tmin, tmax = -0.5, 2
epochs.crop(tmin= tmin, tmax=tmax)
T1 = epochs['T1']
T2 = epochs['T2']
t1 = epochs['T1'].average()
t2 = epochs['T2'].average()
# c =  ['C2','C4','C6', 'C1','C3','C5']
# mne.viz.plot_compare_evokeds([T1,-T2],picks= c);

No projector specified for this dataset. Please consider the method self.add_proj.
Applying baseline correction (mode: mean)
Dropped 1 epoch: 12


In [4]:
epochs

Number of events,14
Events,T1: 7T2: 7
Time range,-0.500 – 2.000 sec
Baseline,-2.000 – -0.200 sec


In [8]:
picks=['C1', 'C2','C3', 'C4', 'C5', 'C6', 'Cz']#,'T9','T10']
epochs['T1'].average().plot_image(titles= 'T1', show_names = 'all',picks = picks);

In [40]:
# epochs.plot_image();
# epochs.plot_psd();
# scale = dict(mag=1e-12, grad=4e-11, eeg=100e-6)
# epochs.plot(scalings = scale);

t1 = epochs['T1'].average()
# t1.plot_joint(times = (0.19,2.47));

t2 = epochs['T2'].average()
# t2.plot_joint(times = (0.19,2.47));

In [ ]:
tmin, tmax = -0.5, 2
epochs_data = epochs.get_data()
# labels = epochs.events[epochs.events[:,-1]!=3]
labels = epochs.events[:,-1]
labels = labels-2
# labels = epochs.events[:,-1]-2
print(f"Data Size: {epochs_data.shape}")

# train data 
epochs_train = epochs.copy().crop(tmin= tmin, tmax=tmax)
epochs_train_data = epochs_train.get_data()
# epochs_train_data = np.transpose(epochs_train_data, axes = (1, 2, 0)) # Sklearn data = n_sam * n_freq
print(f"Train Size: {epochs_train_data.shape}")

In [6]:

picks=['C1', 'C2','C3', 'C4', 'C5', 'C6', 'Cz']#,'T9','T10']
t2.plot_joint(picks=picks);
# t2.plot_image(show_names = 'all', picks = picks);

No projector specified for this dataset. Please consider the method self.add_proj.


In [ ]:
from mne.time_frequency import tfr_morlet
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
frequencies =  np.arange(5,30,1) #np.logspace(*np.log10([5, 30]), num=25)
picks = [1] #baseline = (-0.5, 0.),

In [ ]:
power = tfr_morlet(T1, freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
for pick in picks:
    # power.plot(picks=[pick],  vmin=-0.00003, vmax=0.00003, title=f'{pick}')#, mode='logratio')
    power.plot_joint(timefreqs = [(0.19,10),(2.47,10)],mode = 'logratio', title = f'{list(T1.event_id.keys())[0]}')

In [ ]:
power = tfr_morlet(T2, freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
for pick in picks:
    # power.plot(picks=[pick],  vmin=-0.00003, vmax=0.00003, title=f'{pick}')#, mode='logratio')
    power.plot_joint(timefreqs = [(0.19,10),(2.47,10)],mode = 'logratio', title = f'{list(T2.event_id.keys())[0]}')

In [ ]:
clf = SVC()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(epochs_train_data, labels)
pipe = Pipeline([('scatter', scat_obj), ('clf', clf)])
scores = cross_val_score(pipe, epochs_train_data, labels, cv = cv, verbose=False)